# --- `Day 24`: Arithmetic Logic Unit ---

In [3]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [4]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [5]:
final_input = parse_input(aocd.get_data(day=24, year=2021))
print(final_input[:5])

['inp w', 'mul x 0', 'add x z', 'mod x 26', 'div z 1']


In [6]:
test_input = parse_input('''\
inp x
mul x -1
''')

print(test_input)

['inp x', 'mul x -1']


### Helpers

In [65]:
def readValue(parameter, vars):
    if parameter in ['w', 'x', 'y', 'z']:
        return vars[parameter]
    else:
        return int(parameter)
    
def compute(commands, input):
    vars = {'w':0, 'x':0, 'y':0, 'z':0}
    index = 0
    for line in commands:
        command = line.split(" ")
        if command[0] == "inp":
            vars[command[1]] = int(input[index])
            index += 1
        elif command[0] == "mul":
            vars[command[1]] = vars[command[1]] * readValue(command[2], vars)
        elif command[0] == "add":
            vars[command[1]] = vars[command[1]] + readValue(command[2], vars)
        elif command[0] == "div":
            vars[command[1]] = vars[command[1]] // readValue(command[2], vars)
        elif command[0] == "mod":
            vars[command[1]] = vars[command[1]] % readValue(command[2], vars)
        elif command[0] == "eql":
            result = vars[command[1]] == readValue(command[2], vars)
            vars[command[1]] = 1 if result else 0
            
    if vars['z'] == 0:
        return True
    return False

## Solution 1

In [ ]:
inp w     w: i0, x: 0, y: 0, z: 0
mul x 0   w: i0, x: 0, y: 0, z: 0
add x z   w: i0, x: 0, y: 0, z: 0
mod x 26  w: i0, x: 0, y: 0, z: 0
div z (1)   w: i0, x: 0, y: 0, z: 0
add x (13)  w: i0, x: 13, y: 0, z: 0
eql x w   w: i0, x: 0, y: 0, z: 0
eql x 0   w: i0, x: 1, y: 0, z: 0
mul y 0   w: i0, x: 1, y: 0, z: 0
add y 25  w: i0, x: 1, y: 25, z: 0
mul y x   w: i0, x: 1, y: 25, z: 0
add y 1   w: i0, x: 1, y: 26, z: 0
mul z y   w: i0, x: 1, y: 26, z: 0
mul y 0   w: i0, x: 1, y: 0, z: 0
add y w   w: i0, x: 1, y: i0, z: 0
add y (6)   w: i0, x: 1, y: i0 + 6, z: 0
mul y x   w: i0, x: 1, y: i0 + 6, z: 0
add z y   w: i0, x: 1, y: i0 + 6, z: i0 + 6

z: i0 + 6
                
inp w     w: i1, x: 1, y: i0 + 6, z: i0 + 6
mul x 0   w: i1, x: 0, y: i0 + 6, z: i0 + 6
add x z   w: i1, x: i0 + 6, y: i0 + 6, z: i0 + 6
mod x 26  w: i1, x: (i0 + 6) % 26, y: i0 + 6, z: i0 + 6
div z (1)   w: i1, x: i0 + 6 % 26, y: i0 + 6, z: i0 + 6
add x (15)  w: i1, x: ((i0 + 6) % 26) + 15, y: i0 + 6, z: i0 + 6
eql x w   w: i1, x: 0, y: i0 + 6, z: i0 + 6
eql x 0   w: i1, x: 1, y: i0 + 6, z: i0 + 6
mul y 0   w: i1, x: 1, y: 0, z: i0 + 6
add y 25  w: i1, x: 1, y: 25, z: i0 + 6
mul y x   w: i1, x: 1, y: 25, z: i0 + 6
add y 1   w: i1, x: 1, y: 26, z: i0 + 6
mul z y   w: i1, x: 1, y: 26, z: (i0 + 6) * 26
mul y 0   w: i1, x: 1, y: 0, z: (i0 + 6) * 26
add y w   w: i1, x: 1, y: i1, z: (i0 + 6) * 26
add y (7)   w: i1, x: 1, y: i1 + 7, z: (i0 + 6) * 26
mul y x   w: i1, x: 1, y: (i1 + 7), z: (i0 + 6) * 26
add z y   w: i1, x: 1, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)

z: ((i0 + 6) * 26) + (i1 + 7)
                
inp w     w: i2, x: 1, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
mul x 0   w: i2, x: 0, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
add x z   w: i2, x: ((i0 + 6) * 26) + (i1 + 7), y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
mod x 26  w: i2, x: (((i0 + 6) * 26) + (i1 + 7)) % 26, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
div z (1)   w: i2, x: (((i0 + 6) * 26) + (i1 + 7)) % 26, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
add x (15) w: i2, x: (((i0 + 6) * 26) + (i1 + 7)) % 26 + 15, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)  
eql x w   w: i2, x: 0, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)  
eql x 0   w: i2, x: 1, y: (i1 + 7), z: ((i0 + 6) * 26) + (i1 + 7)
mul y 0   w: i2, x: 1, y: 0, z: ((i0 + 6) * 26) + (i1 + 7)
add y 25  w: i2, x: 1, y: 25, z: ((i0 + 6) * 26) + (i1 + 7)
mul y x   w: i2, x: 1, y: 25, z: ((i0 + 6) * 26) + (i1 + 7)
add y 1   w: i2, x: 1, y: 26, z: ((i0 + 6) * 26) + (i1 + 7)
mul z y   w: i2, x: 1, y: 26, z: (((i0 + 6) * 26) + (i1 + 7)) * 26
mul y 0   w: i2, x: 1, y: 0, z: (((i0 + 6) * 26) + (i1 + 7)) * 26
add y w   w: i2, x: 1, y: i2, z: (((i0 + 6) * 26) + (i1 + 7)) * 26
add y (10)  w: i2, x: 1, y: i2 + 10, z: (((i0 + 6) * 26) + (i1 + 7)) * 26
mul y x   w: i2, x: 1, y: i2 + 10, z: (((i0 + 6) * 26) + (i1 + 7)) * 26
add z y   w: i2, x: 1, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)

z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
                
inp w     w: i3, x: 1, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul x 0   w: i3, x: 0, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add x z   w: i3, x: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10), y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mod x 26  w: i3, x: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) % 26, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
div z (1)  w: i3, x: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) % 26, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10) 
add x (11)  w: i3, x: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) % 26) + 11, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
eql x w   w: i3, x: 0, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
eql x 0   w: i3, x: 1, y: i2 + 10, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y 0   w: i3, x: 1, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y 25  w: i3, x: 1, y: 25, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y x   w: i3, x: 1, y: 25, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y 1   w: i3, x: 1, y: 26, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul z y   w: i3, x: 1, y: 26, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26
mul y 0   w: i3, x: 1, y: 0, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26
add y w   w: i3, x: 1, y: i3, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26
add y (2) w: i3, x: 1, y: i3 + 2, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26  
mul y x   w: i3, x: 1, y: i3 + 2, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26  
add z y   w: i3, x: 1, y: i3 + 2, z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)
    
z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)

###########################################################
inp w     w: i4, x: 1, y: i3 + 2, z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)
mul x 0   w: i4, x: 0, y: i3 + 2, z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)
add x z   w: i4, x: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2), y: i3 + 2, z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)
mod x 26  w: i4, x: (i3 + 2), y: i3 + 2, z: ((((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)) * 26) + (i3 + 2)
div z (26)  w: i4, x: (i3 + 2), y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add x (-7)  w: i4, x: (i3 + 2) - 7, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
eql x w   w: i4, x: ((i3 + 2) - 7) == i4, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
                
((i3 + 2) - 7) != i4
eql x w   w: i4, x: 0, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
eql x 0   w: i4, x: 1, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y 0   w: i4, x: 1, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7))
add y 25  w: i4, x: 1, y: 25, z: ((((i0 + 6) * 26) + (i1 + 7))
mul y x   w: i4, x: 1, y: 25, z: ((((i0 + 6) * 26) + (i1 + 7))
add y 1   w: i4, x: 1, y: 26, z: ((((i0 + 6) * 26) + (i1 + 7))
mul z y   w: i4, x: 1, y: 26, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26
mul y 0   w: i4, x: 1, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26
add y w   w: i4, x: 1, y: i4, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26
add y (15) w: i4, x: 1, y: i4 + 15, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26
mul y x   w: i4, x: 1, y: i4 + 15, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26
add z y   w: i4, x: 1, y: i4 + 15, z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i4 + 15)

z: (((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i4 + 15)
this is bad, z still getting bigger
                                       
((i3 + 2) - 7) == i4
eql x w   w: i4, x: 1, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
eql x 0   w: i4, x: 0, y: i3 + 2, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y 0   w: i4, x: 0, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y 25  w: i4, x: 0, y: 25, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y x   w: i4, x: 0, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y 1   w: i4, x: 0, y: 1, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul z y   w: i4, x: 0, y: 1, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y 0   w: i4, x: 0, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y w   w: i4, x: 0, y: i4, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add y (15) w: i4, x: 0, y: i4 + 15, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
mul y x   w: i4, x: 0, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
add z y   w: i4, x: 0, y: 0, z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)

z: ((((i0 + 6) * 26) + (i1 + 7)) * 26) + (i2 + 10)
this one good, lost bottom part of z, i3.  didn't add i4
need ((i3 + 2) - 7) == i4
    i4 = i3 - 5
    
look for pattern, each section repeats. result depends on () numbers
############################################################
z = [i0 + 6, i1 + 7, i2 + 10]    

# w = getInput
# x = z % 26 + b
# z //= a
# if x != w:
#   z *= 26
#   z += (w + c)
    
w = i5
z *= 26
z += (i5 + 8)
    
z = [i0 + 6, i1 + 7, i2 + 10, i5 + 8]
    
w = i6
z *= 26
z += (i6 + 1)
    
z = [i0 + 6, i1 + 7, i2 + 10, i5 + 8, i6 + 1]
    
w = i7
x = z % 26 - 5
z //= 26
if x != i7:
  z *= 26
  z += (i7 + 10)
    
i7 = (i6 + 1) - 5 = i6 - 4
z = [i0 + 6, i1 + 7, i2 + 10, i5 + 8]

In [67]:
#inp w
#mul x 0
#add x z
#mod x 26
#div z (26)  -> a
#add x (-9)  -> b
#eql x w
#eql x 0
#mul y 0
#add y 25
#mul y x
#add y 1
#mul z y
#mul y 0
#add y w
#add y (12)  -> c
#mul y x
#add z y

# all blocks are same except () numbers
# first is 1 or 26
# for second, if first 1 range 10 - 15
#                      2 range -9 - 0
# for third range 1 - 15

# w = getInput
# x = 0
# x += z
# x %= 26
# z //= a
# x += b
# x = 1 if x == w else 0
# x = 1 if x == 0 else 0
# y = 0
# y += 25
# y *= x
# y += 1
# z *= y
# y = 0
# y += w
# y += c
# y *= x
# z += y

# w = getInput
# x = z % 26 + b
# z //= a
# x = 1 if x != w else 0
# y = 25 * x + 1
# z *= y
# z += (w + c) * x

# w = getInput
# x = z % 26 + b
# z //= a
# if x != w:
#   z *= 26
#   z += (w + c)

# if a is 1, new z value (w + c) pushed, b larger than 9 so x != w
# if a is 26, z value popped, need to avoid pushing new one back
#     z % 26 + b needs to equal w
#     z % 26 is last thing pushed
# it is like adding new digit to base 26 numbers
# 7 pushes and 7 pops need to equal out to end at z = zero

# from input file (a, b, c)
parameters = [
    [1,13,6],    # push i0 + 6
    [1,15,7],    # push i1 + 7
    [1,15,10],   # push i2 + 10
    [1,11,2],    # push i3 + 2
    [26,-7,15],  # pop i4 == popped - 7
    [1,10,8],    # push i5 + 8
    [1,10,1],    # push i6 + 1
    [26,-5,10],  # pop i7 == popped - 5
    [1,15,5],    # push i8 + 5
    [26,-3,3],   # pop i9 == popped - 3
    [26,0,5],    # pop i10 == popped
    [26,-5,11],  # pop i11 == popped - 5
    [26,-9,12],  # pop i12 == popped - 9
    [26,0,10]    # pop i13 == popped - 0
]

# i4 = i3 - 5        (-7 + 2)
# i7 = i6 - 4        (-5 + 1)
# i9 = i8 + 2        (-3 + 5)
# i10 = i5 + 8       (0 + 8)
# i11 = i2 + 5       (-5 + 10)
# i12 = i1 - 2       (-9 + 7)
# i13 = i0 + 6       (0 + 6)

# largest
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13
# 3 9 4 9 4 1 9 5 7 9 9  9  7  9

# smallest
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13
# 1 3 1 6 1 1 5 1 1 3 9  6  1  7
    
def solve_1(input):
    for i in [39494195799979, 13161151139617]:
        s = str(i)
        if compute(input, str(i)):
            print(i, "good")
        else:
            print(i, "bad")

solve_1(test_input)
#33161151139619 too high

39494195799979 good
13161151139617 good
